<a href="https://colab.research.google.com/github/zuhouralsaqa/zuhouralsaqa/blob/main/MountainCar_v0_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. dependences

In [2]:
!apt-get update
!apt-get install git

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,180 kB]
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,119 kB]
Fetched 3,655 kB in 6s (6

In [3]:
!git clone git://github.com/vanderlin/ofxBox2d.git
!cd ofxBox2d
!git checkout 008

Cloning into 'ofxBox2d'...
fatal: unable to connect to github.com:
github.com[0: 140.82.114.4]: errno=Connection timed out

/bin/bash: line 1: cd: ofxBox2d: No such file or directory
fatal: not a git repository (or any of the parent directories): .git


In [4]:
!apt-get -qq -y install swig> /dev/null
!apt-get install build-essential python-dev swig python-pygame subversion > /dev/null
!git clone

E: Package 'python-dev' has no installation candidate
E: Unable to locate package python-pygame
fatal: You must specify a repository to clone.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
             

In [5]:
!apt-get install pygame

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package pygame


In [6]:
!pip install --upgrade pip

In [7]:
from google.colab.patches import cv2_imshow
import cv2
import google.colab.patches as patches
import gym
import IPython.display as display
import time

In [8]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender
!pip install imageio==2.5
!pip install --upgrade AutoROM
!AutoROM --accept-license
!pip install gym[atari,accept-rom-license]

  and should_run_async(code)



  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
  Attempting uninstall: AutoROM
    Found existing installation: AutoROM 0.4.2
    Uninstalling AutoROM-0.4.2:
      Successfully uninstalled AutoROM-0.4.2


AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.
  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1


In [9]:
!pip install renderlab

In [10]:
import random
import gym
import numpy as np
from collections import deque
import pygame
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os

# 2. the code starts here

In [15]:
env = gym.make('MountainCar-v0', render_mode='human')
(state) = env.reset()
timeStep=1000
batch_size = 50
n_episodes = 6000

In [16]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [17]:
output_dir = 'model_output/MountainCar-v0/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [18]:
class DQNAgent:
    def __init__(self, state_size, action_size, init_epsilon):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95

        self.epsilon = init_epsilon

        self.epsilon_decay = 0.9992

        self.epsilon_min = 0.01

        self.learning_rate = 0.001

        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            target = reward

            if not (done or terminated):
                target = (reward + self.gamma *np.amax(self.model.predict(next_state)[0]))
                target_f = self.model.predict(state)
                target_f[0][action] = target
                self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

    def newreward(self, pos, speed):
        if (pos >= 0.5):
            return 1000
        else:
            return (((pos+1.2)**3/5.832)+((speed**2)/0.0049))*2 - 1


# 3. load data

***load your files***

---



In [14]:
# Check if the file exists in the current directory
import os
os.listdir()

['.config', 'model_output', 'my_memory_data.pkl', 'sample_data']

In [15]:
import pickle

# Specify the path to the file you want to load
file_path = 'my_memory_data.pkl'

# Load the variables from the file
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)

# Now, 'loaded_data' contains the variables you saved
print(loaded_data)

{'example_variable': deque([(array([[-0.7687723 , -0.00618698]], dtype=float32), 0, -0.9611508110208092, array([[-0.77428186, -0.00550955]], dtype=float32), False), (array([[-0.77428186, -0.00550955]], dtype=float32), 0, -0.9650151360673196, array([[-0.77908355, -0.00480171]], dtype=float32), False), (array([[-0.77908355, -0.00480171]], dtype=float32), 0, -0.968406517049854, array([[-0.78315127, -0.00406774]], dtype=float32), False), (array([[-0.78315127, -0.00406774]], dtype=float32), 0, -0.9712704830045245, array([[-0.7864632 , -0.00331193]], dtype=float32), False), (array([[-0.7864632 , -0.00331193]], dtype=float32), 0, -0.9735612292326213, array([[-0.78900176, -0.00253853]], dtype=float32), False), (array([[-0.78900176, -0.00253853]], dtype=float32), 0, -0.975242104825846, array([[-0.7907535 , -0.00175175]], dtype=float32), False), (array([[-0.7907535 , -0.00175175]], dtype=float32), 0, -0.9762860305080858, array([[-0.7917093 , -0.00095581]], dtype=float32), False), (array([[-0.791

In [29]:
agent = DQNAgent(state_size, action_size, 0.060257767174608595)
agent.load(output_dir + 'weights_0050' + '.hdf5')
init_e = 2100

  super().__init__(name, **kwargs)



In [17]:
# Access a specific variable from the loaded data
agent.memory = loaded_data['example_variable']

# Use the loaded variable in your code
print(f"Loaded variable: {agent.memory}")

Loaded variable: deque([(array([[-0.7687723 , -0.00618698]], dtype=float32), 0, -0.9611508110208092, array([[-0.77428186, -0.00550955]], dtype=float32), False), (array([[-0.77428186, -0.00550955]], dtype=float32), 0, -0.9650151360673196, array([[-0.77908355, -0.00480171]], dtype=float32), False), (array([[-0.77908355, -0.00480171]], dtype=float32), 0, -0.968406517049854, array([[-0.78315127, -0.00406774]], dtype=float32), False), (array([[-0.78315127, -0.00406774]], dtype=float32), 0, -0.9712704830045245, array([[-0.7864632 , -0.00331193]], dtype=float32), False), (array([[-0.7864632 , -0.00331193]], dtype=float32), 0, -0.9735612292326213, array([[-0.78900176, -0.00253853]], dtype=float32), False), (array([[-0.78900176, -0.00253853]], dtype=float32), 0, -0.975242104825846, array([[-0.7907535 , -0.00175175]], dtype=float32), False), (array([[-0.7907535 , -0.00175175]], dtype=float32), 0, -0.9762860305080858, array([[-0.7917093 , -0.00095581]], dtype=float32), False), (array([[-0.7917093

# 4. training my agent

In [ ]:
# agent = DQNAgent(state_size, action_size)
done = False
terminated = False
wins, episode = 0, 0
n_episodes = init_e + 151
for e in range(init_e, n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    terminated = False
    end = done or terminated
    tot_reward, reward = 0, 0

    while not done:

        env.render()

        action = agent.act(state)
        next_state, reward, done, info = env.step(action)
        position = next_state[0]
        velocity = next_state[1]
        reward = agent.newreward(position, velocity)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        tot_reward += reward

        if done :
            print("episode: {}/{}, score: {}, e: {: .2}".format(e, n_episodes-1, tot_reward, agent.epsilon))
            if tot_reward > 100:
              wins += 1
              print("You Win")
            episode += 1
            print(f"wins:{wins}/{episode}")
            break

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:05d}'.format(e) + ".hdf5")

pygame.quit()


# 5. save data

In [19]:
agent.save(output_dir + "weights_" + '{:05d}'.format(e) + ".hdf5")

In [20]:
agent.epsilon

0.05339852554333118

In [21]:
wins

122

In [22]:
save_memory = agent.memory

In [23]:
import pickle

# Your data to be saved
data = {'example_variable': save_memory}

# Save data to a file in the Colab environment
with open('my_memory_data.pkl', 'wb') as file:
    pickle.dump(data, file)


# 6. test


In [ ]:
import gymnasium as gym
import renderlab as rl

env = gym.make("MountainCar-v0", render_mode = "rgb_array")
env = rl.RenderFrame(env, "./output")

observation, info = env.reset()

while True:
  action = env.action_space.sample()
  observation, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    break

env.play()

In [113]:
agent.load(output_dir + 'weights_02150' + '.hdf5')

In [114]:
import gymnasium as gym
import renderlab as rl

env = gym.make("MountainCar-v0", render_mode = "rgb_array")
env = rl.RenderFrame(env, "./output")

state, _ = env.reset()
state = np.reshape(state, [1, state_size])


while True:
    act_values = agent.model.predict(state)
    action = np.argmax(act_values[0])
    next_state, reward, terminated, truncated, info = env.step(action)
    next_state = np.reshape(next_state, [1, state_size])
    state = next_state

    if terminated or truncated:
      break

env.play()

1/1 [==============================] - 0s 48ms/step
Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4
